In [10]:
from openai import OpenAI
import pandas as pd
import json
import pickle
from tqdm.auto import tqdm
import requests
from sklearn.model_selection import GroupKFold
import numpy as np
import glob
import torch
import pickle
import time
import random
def request_openai_chatgpt_version(message, system, temperature=1., top_p=0.8,model_name ="gpt-4o-2024-05-13"):
    pass
    ### your openai code

In [11]:
path_prefix = "../data"

In [12]:
train = pd.read_csv(f"{path_prefix}/train.csv")
test = pd.read_csv(f"{path_prefix}/test.csv")
sample_submission = pd.read_csv(f"{path_prefix}/sample_submission.csv")
misconception_mapping = pd.read_csv(f"{path_prefix}/misconception_mapping.csv")

In [13]:
groups = train['QuestionId'].values
# 创建 GroupKFold 对象
group_kfold = GroupKFold(n_splits=5)
train = train.reset_index(drop=True)
# 进行分组交叉验证
for train_index, test_index in group_kfold.split(train, groups=groups):
    tra = train.iloc[train_index,:]
    val = train.iloc[test_index,:]
    # tra = train
    break
tra.shape,val.shape

((1495, 15), (374, 15))

In [14]:
misconception_mapping_dict = {}
for _,row in misconception_mapping.iterrows():
    misconception_mapping_dict[row['MisconceptionId']] = row['MisconceptionName']

In [15]:
misconception_mapping['MisconceptionName'].apply(lambda x: len(x.split(' '))).describe()

count    2587.000000
mean       12.153460
std         5.241945
min         2.000000
25%         8.000000
50%        11.000000
75%        15.000000
max        35.000000
Name: MisconceptionName, dtype: float64

In [16]:
examples = []
for _,row in tra.iterrows():
    for c in ['A','B','C','D']:
        if str(row[f"Misconception{c}Id"])!="nan":
            reason=misconception_mapping_dict[row[f"Misconception{c}Id"]]
            real_answer_id = row['CorrectAnswer']
            real_text = row[f'Answer{real_answer_id}Text']
            examples.append(f"SubjectName:{row['SubjectName']}\nConstructName:{row['ConstructName']}\nQuestion:{row['QuestionText']}\nCandidateAnswer:A.{row['AnswerAText']}\nB.{row['AnswerBText']}\nC.{row['AnswerCText']}\nD.{row['AnswerDText']}\nSelectedAnswer:{c}.{row[f'Answer{c}Text']}\nCorrectAnswer:{real_answer_id}.{real_text}\nMisconceptionReason:{reason}")

In [17]:
example_str = examples[2] +"#$%" +examples[6] + "#$%" + examples[12]
system_prompt = f"""You are a mathematician, skilled in various mathematical operations and calculations.
Generate similar samples based on the given examples, ensuring there are no contradictions and they are completely logical.
Example:
{example_str}
"""
prompt = f"""Now given a misconception reason, please generate three different types of samples that meet this misconception reason, ensuring that this is the only reason leading to the incorrect choice. Separate each sample with '#$%'.
Misconception Reason:{misconception_mapping_dict[28]}"""

In [18]:
example_str

"SubjectName:Simplifying Algebraic Fractions\nConstructName:Simplify an algebraic fraction by factorising the numerator\nQuestion:Simplify the following, if possible: \\( \\frac{m^{2}+2 m-3}{m-3} \\)\nCandidateAnswer:A.\\( m+1 \\)\nB.\\( m+2 \\)\nC.\\( m-1 \\)\nD.Does not simplify\nSelectedAnswer:B.\\( m+2 \\)\nCorrectAnswer:D.Does not simplify\nMisconceptionReason:Thinks that when you cancel identical terms from the numerator and denominator, they just disappear#$%SubjectName:Range and Interquartile Range from a List of Data\nConstructName:Calculate the range from a list of data\nQuestion:Tom and Katie are discussing the \\( 5 \\) plants with these heights:\n\\( 24 \\mathrm{~cm}, 17 \\mathrm{~cm}, 42 \\mathrm{~cm}, 26 \\mathrm{~cm}, 13 \\mathrm{~cm} \\)\nTom says if all the plants were cut in half, the range wouldn't change.\nKatie says if all the plants grew by \\( 3 \\mathrm{~cm} \\) each, the range wouldn't change.\nWho do you agree with?\nCandidateAnswer:A.Only\nTom\nB.Only\nKatie

In [8]:
res = request_openai_chatgpt_version(prompt, system_prompt, temperature=0.1, top_p=0.1,model_name = "gpt-4o-2024-08-06")

In [9]:
res

('#$%SubjectName:Interior Angles of Polygons\nConstructName:Calculate the sum of interior angles of a polygon\nQuestion:What is the sum of the interior angles of a hexagon?\nCandidateAnswer:A.\\( 720^\\circ \\)\nB.\\( 540^\\circ \\)\nC.\\( 360^\\circ \\)\nD.\\( 180^\\circ \\)\nSelectedAnswer:C.\\( 360^\\circ \\)\nCorrectAnswer:A.\\( 720^\\circ \\)\nMisconceptionReason:Thinks the interior angles of any polygon add up to 360#$%SubjectName:Interior Angles of Polygons\nConstructName:Identify the sum of interior angles of a polygon\nQuestion:Calculate the sum of the interior angles of a pentagon.\nCandidateAnswer:A.\\( 540^\\circ \\)\nB.\\( 360^\\circ \\)\nC.\\( 720^\\circ \\)\nD.\\( 180^\\circ \\)\nSelectedAnswer:B.\\( 360^\\circ \\)\nCorrectAnswer:A.\\( 540^\\circ \\)\nMisconceptionReason:Thinks the interior angles of any polygon add up to 360#$%SubjectName:Interior Angles of Polygons\nConstructName:Determine the sum of interior angles of a polygon\nQuestion:Find the sum of the interior a

In [ ]:

for mp_id,mis_value in tqdm(misconception_mapping_dict.items()):
    sample_example = random.sample(examples, 3)
    example_str = sample_example[0] +"#$%" +sample_example[1] + "#$%" + sample_example[2]
    system_prompt = f"""You are a mathematician, skilled in various mathematical operations and calculations.
Generate similar samples based on the given examples, ensuring there are no contradictions and they are completely logical.
Example:
{example_str}
"""
    prompt = f"""Now given a misconception reason, please generate three different types of samples that meet this misconception reason, ensuring that this is the only reason leading to the incorrect choice. Separate each sample with '#$%'.
    Misconception Reason:{mis_value}"""
    res = request_openai_chatgpt_version(prompt, system_prompt, temperature=0.01, top_p=0.01,model_name = "gpt-4o-2024-08-06")
    if res[0] is None:
        continue
    save_time  = int(time.time())
    with open(f"./save_data/cv1/{mp_id}_{save_time}.pkl",'wb') as f:
        pickle.dump([system_prompt,prompt,res],f)

 64%|██████▎   | 1647/2587 [4:50:50<6:42:44, 25.71s/it]

In [12]:
mp_id

2586